In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
import sys
sys.path.append("../camp_scripts") # change accordingly
sns.set_context('notebook')

In [4]:
from june.demography.geography import Geography
from june.demography.demography import load_age_and_sex_generators_for_bins, Demography, Population
from june.paths import data_path
try:
    from june.paths import camp_data_path
    data_path = camp_data_path
except:
    camp_data_path = data_path
from june import World
from june.groups import Hospital, Hospitals
from june.distributors import HospitalDistributor
from june.world import generate_world_from_hdf5
from camp_creation import generate_empty_world, populate_world, distribute_people_to_households # this is loaded from the ../camp_scripts folder

No --data argument given - defaulting to:
/home/arnau/code/CPModelling/data
No --camp_data argument given - defaulting to:
/home/arnau/code/CPModelling/camp_data
No --configs argument given - defaulting to:
/home/arnau/code/CPModelling/configs


In [5]:
from june.groups.leisure import PumpLatrines, PumpLatrineDistributor
from june.groups.leisure import DistributionCenters, DistributionCenterDistributor
from june.groups.leisure import Communals, CommunalDistributor

## Initialize the world

In [6]:
# create empty world's geography
world = generate_empty_world(filter_key = {"region" : ["CXB-219"]})

2020-06-10 15:44:44,217 - june.demography.geography - INFO - There are 65 areas and 4 super_areas in the world.


In [7]:
# populate empty world
populate_world(world)

In [8]:
# distribute people to households
distribute_people_to_households(world)

In [9]:
hospitals= Hospitals.from_file(
    filename=camp_data_path / 'input/hospitals/hospitals.csv'
)
world.hospitals = hospitals
hospital_distributor = HospitalDistributor(hospitals, 
                                           medic_min_age=20,
                                           patients_per_medic=10)
hospital_distributor.distribute_medics_from_world(world.people)


2020-06-10 15:44:45,316 - june.groups.hospital - INFO - There are 8 hospitals in the world.


In [11]:
world.pumplatrines = PumpLatrines.for_areas(world.areas)
world.distribution_centers = DistributionCenters.for_areas(world.areas)
world.communal = Communals.for_areas(world.areas)

FileNotFoundError: [Errno 2] File /home/arnau/code/CPModelling/camp_data/input/activities/distribution_centers.csv does not exist: '/home/arnau/code/CPModelling/camp_data/input/activities/distribution_centers.csv'

## Geography exploration

### UNHCR camp blocks inside the World

In [ ]:
super_areas = []
for super_area in world.super_areas:
    super_areas.append(super_area.name)

In [ ]:
super_areas[:5]

### Mahjee blocks inside World

In [ ]:
areas = []
for area in world.areas:
    areas.append(area.name)

In [ ]:
areas[:5]

## Demography exploration

### UNHCR block residents

In [ ]:
sample_ages = []
sample_sexes = []
for person in world.super_areas.members[0].people:
    sample_ages.append(person.age)
    sample_sexes.append(person.sex)

In [ ]:
sample_sexes[:5], sample_ages[:5]

In [ ]:
ages, counts = np.unique(sample_ages, return_counts=True)
plt.bar(ages, counts)
plt.title('super area: {} age distribution'.format(world.super_areas.members[0].name))
plt.show()

In [ ]:
print ('The % children between ages 0-16 is {}%'.format(round(np.sum(counts[:16])/(np.sum(counts[:16])+np.sum(counts[16:]))*100)))

## Households

### Household size distribution

In [ ]:
household_sizes = [household.size for household in world.households]
plt.hist(household_sizes, density=True, bins=np.arange(0,12))
plt.xlabel("Household size")
plt.ylabel("Frequency")
plt.axvline(np.mean(household_sizes), color="cyan")
plt.text(8, 0.150, f"Avg. size: {(np.mean(household_sizes)):.2f}")
plt.show()

## Activity exploration

### Pumps and latrines 

In [ ]:
pumps_and_latrines = []
for pump_latrine in world.areas.members[0].pumplatrines:
    pumps_and_latrines.append(pump_latrine)

In [ ]:
print ('There are {} pumps and latrines in area {}'.format(len(pumps_and_latrines), world.areas.members[0].name))

### Distribution centers

In [ ]:
distribution_centers = []
for center in world.distribution_centers:
    distribution_centers.append(center)

In [ ]:
distribution_centers[0].coordinates

### Communal spaces

In [ ]:
communal_spaces = []
for space in world.communal:
    communal_spaces.append(space)

In [ ]:
communal_spaces[0].coordinates